In [ ]:
import requests
import mysql.connector
import json
import os

# API configuration
API_KEY = "afC1JKkLVgs2OqmVsFsor13vsmNoN3jihUct7H8L"
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json"
url = f"{BASE_URL}?api_key={API_KEY}"
headers = {"accept": "application/json"}

try:
    # Step 1: Fetch data from API
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    # Save to competitions.json (optional)
    with open("competitions.json", "w") as f:
        json.dump(data, f, indent=4)
    print("Data fetched and saved to competitions.json")

    # Step 2: Connect to MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Padmavathi@09",
        database="tennis_db",
        auth_plugin='mysql_native_password'
    )
    cursor = conn.cursor()

    # Step 3: Debug - Print Existing Tables
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    print("Existing tables in database:", tables)

    # Step 4: Create 'Categories' table if not exists (without country_code)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Categories (
            category_id VARCHAR(50) PRIMARY KEY,
            category_name VARCHAR(100)
        )
    """)
    print("Table 'Categories' ensured.")

    # Step 5: Create 'Competitions' table if not exists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Competitions (
            competition_id VARCHAR(50) PRIMARY KEY,
            competition_name VARCHAR(100),
            type VARCHAR(50),
            gender VARCHAR(20),
            category_id VARCHAR(50),
            FOREIGN KEY (category_id) REFERENCES Categories(category_id)
        )
    """)
    print("Table 'Competitions' ensured.")

    # Step 6: Insert data into 'Categories' table (if not exists)
    for item in data['competitions']:
        category = item["category"]
        if "id" in category and "name" in category:  # Ensure keys exist
            cursor.execute("""
                INSERT IGNORE INTO Categories (category_id, category_name) 
                VALUES (%s, %s)
            """, (category["id"], category["name"]))

    # Step 7: Insert data into 'Competitions' table
    for item in data['competitions']:
        cursor.execute("""
            INSERT IGNORE INTO Competitions (competition_id, competition_name, type, gender, category_id) 
            VALUES (%s, %s, %s, %s, %s)
        """, (item["id"], item["name"], item["type"], item["gender"], item["category"]["id"]))

    # Step 8: Commit changes
    conn.commit()
    print("Data imported into MySQL successfully!")

except requests.exceptions.RequestException as e:
    print(f"API error: {e}")
except mysql.connector.Error as e:
    print(f"Database error: {e}")
except KeyError as e:
    print(f"Missing key in API data: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("Database connection closed.")

In [ ]:
import requests
import mysql.connector
import json

# API configuration
API_KEY = "afC1JKkLVgs2OqmVsFsor13vsmNoN3jihUct7H8L"
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"
url = f"{BASE_URL}?api_key={API_KEY}"

try:
    # Step 1: Fetch data from API
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Step 2: Save to JSON for debugging
    with open("complexes.json", "w") as f:
        json.dump(data, f, indent=4)
    print("Data fetched and saved to complexes.json")

    # Step 3: Connect to MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Padmavathi@09",
        database="tennis_db",
        auth_plugin='mysql_native_password'
    )
    cursor = conn.cursor()

    # Step 4: Drop and recreate tables to ensure clean schema (optional: remove if keeping old data)
    cursor.execute("DROP TABLE IF EXISTS Venues, Complexes")
    cursor.execute("""
        CREATE TABLE Complexes (
            complex_id VARCHAR(50) PRIMARY KEY,
            complex_name VARCHAR(100) NOT NULL
        )
    """)
    cursor.execute("""
        CREATE TABLE Venues (
            venue_id VARCHAR(50) PRIMARY KEY,
            venue_name VARCHAR(100) NOT NULL,
            city_name VARCHAR(100) NOT NULL DEFAULT 'Unknown',
            country_name VARCHAR(100) NOT NULL DEFAULT 'Unknown',
            country_code CHAR(3) NOT NULL DEFAULT 'UNK',
            timezone VARCHAR(100) NOT NULL DEFAULT 'Unknown',
            complex_id VARCHAR(50),
            FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
        )
    """)
    print("Tables 'Complexes' and 'Venues' recreated.")

    # Step 5: Insert data into 'Complexes'
    complex_count = 0
    for complex_item in data.get('complexes', []):
        cursor.execute("""
            INSERT IGNORE INTO Complexes (complex_id, complex_name) 
            VALUES (%s, %s)
        """, (complex_item["id"], complex_item["name"]))
        complex_count += cursor.rowcount

    # Step 6: Insert data into 'Venues'
    venue_count = 0
    for complex_item in data.get('complexes', []):
        for venue in complex_item.get("venues", []):
            venue_id = venue.get("id")
            venue_name = venue.get("name", "Unknown")
            city_name = venue.get("city_name", "Unknown")  # Matches API key
            country_name = venue.get("country_name", "Unknown")  # Matches API key
            country_code = venue.get("country_code", "UNK")
            timezone = venue.get("timezone", "Unknown")
            complex_id = complex_item.get("id")

            # Debug: Print before insertion
            print(f"Inserting venue: ID={venue_id}, Name={venue_name}, City={city_name}, "
                  f"Country={country_name}, Code={country_code}, Timezone={timezone}, Complex={complex_id}")

            cursor.execute("""
                INSERT IGNORE INTO Venues 
                (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id))
            venue_count += cursor.rowcount

    # Step 7: Commit changes
    conn.commit()
    print(f"Data imported successfully! Inserted {complex_count} complexes and {venue_count} venues.")

    # Step 8: Verify data
    cursor.execute("SELECT venue_id, venue_name, city_name, country_name FROM Venues LIMIT 5")
    rows = cursor.fetchall()
    print("Sample data from Venues:")
    for row in rows:
        print(row)

except requests.exceptions.RequestException as e:
    print(f"API error: {e}")
except mysql.connector.Error as e:
    print(f"Database error: {e}")
except KeyError as e:
    print(f"Missing key in API data: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("Database connection closed.")

In [ ]:
import requests
import mysql.connector
import json

# API configuration
API_KEY = "afC1JKkLVgs2OqmVsFsor13vsmNoN3jihUct7H8L"
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en/rankings.json"
url = f"{BASE_URL}?api_key={API_KEY}"
headers = {"accept": "application/json"}

try:
    # Step 1: Fetch data from API
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    # Save to rankings.json (optional for debugging)
    with open("rankings.json", "w") as f:
        json.dump(data, f, indent=4)
    print("✅ Data fetched and saved to rankings.json")

    # Step 2: Connect to MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Padmavathi@09",
        database="tennis_db",
        auth_plugin='mysql_native_password'
    )
    cursor = conn.cursor()

    # Step 3: Ensure tables exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Competitors (
            competitor_id VARCHAR(50) PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            country VARCHAR(100) NOT NULL,
            country_code CHAR(3) NOT NULL,
            abbreviation VARCHAR(10) NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Competitor_Rankings (
            rank_id INT PRIMARY KEY AUTO_INCREMENT,
            `rank` INT NOT NULL,
            movement INT NOT NULL,
            points INT NOT NULL,
            competitions_played INT NOT NULL,
            competitor_id VARCHAR(50),
            FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id)
        )
    """)
    print("✅ Tables 'Competitors' and 'Competitor_Rankings' ensured.")

    # Step 4: Insert data into 'Competitors'
    competitor_count = 0
    ranking_count = 0

    for ranking in data.get("rankings", []):
        for competitor_data in ranking.get("competitor_rankings", []):
            competitor = competitor_data.get("competitor", {})

            competitor_id = competitor.get("id")
            name = competitor.get("name", "Unknown")
            country = competitor.get("country", "Unknown")
            country_code = competitor.get("country_code", "UNK")
            abbreviation = competitor.get("abbreviation", "UNK")

            # Insert into Competitors table (ignores duplicates)
            cursor.execute("""
                INSERT IGNORE INTO Competitors (competitor_id, name, country, country_code, abbreviation) 
                VALUES (%s, %s, %s, %s, %s)
            """, (competitor_id, name, country, country_code, abbreviation))
            competitor_count += cursor.rowcount

            # Insert into Competitor_Rankings table
            cursor.execute("""
                INSERT INTO Competitor_Rankings (`rank`, movement, points, competitions_played, competitor_id) 
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                `rank` = VALUES(`rank`), movement = VALUES(movement), 
                points = VALUES(points), competitions_played = VALUES(competitions_played)
            """, (
                competitor_data["rank"], 
                competitor_data["movement"], 
                competitor_data["points"], 
                competitor_data["competitions_played"], 
                competitor_id
            ))
            ranking_count += cursor.rowcount

    # Step 5: Commit changes
    conn.commit()
    print(f"✅ Data inserted successfully! {competitor_count} competitors and {ranking_count} rankings added.")

    # Step 6: Verify data
    cursor.execute("SELECT * FROM Competitor_Rankings")
    rows = cursor.fetchall()
    print("🔍 Sample data from Competitor_Rankings:")
    for row in rows:
        print(row)

except requests.exceptions.RequestException as e:
    print(f"❌ API error: {e}")
except mysql.connector.Error as e:
    print(f"❌ Database error: {e}")
except KeyError as e:
    print(f"❌ Missing key in API data: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("🔒 Database connection closed.")
